### LSTM으로 텍스트 생성하기

In [1]:
# 다른 온도 값을 사용하여 확률 분포의 가중치 바꾸기
import numpy as np

# original_distribution은 전체 합이 1인 1D 넘파이 배열
# temperature는 출력 분포의 엔트로피 야을 결정
def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution) # 원본 분포의 가중치를 변경하며 반환 / 이 분포의 합은 1이 아닐 수 있으므로 새로운 분포의 합으로 나눔

In [2]:
# 원본 텍스트 파일을 내려받아 파싱하기
import keras
import numpy as np

path = keras.utils.get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('말뭉치 크기:', len(text))

Using TensorFlow backend.
c:\users\rladh\anaconda3\envs\py36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\rladh\anaconda3\envs\py36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\rladh\anaconda3\envs\py36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\rladh\anaconda3\envs\py36\lib\site-packag

606208/600901 [==============================] - 4s 7us/step
말뭉치 크기: 600893


In [4]:
# 글자 시퀀스 벡터화하기
maxlen = 60 # 60개의 글자로 된 시퀀스를 추출
step = 3 # 세 글자씩 건너뛰면서 새로운 시퀀스를 샘플링

sentences = [] # 추출한 시퀀스를 담을 리스트

next_chars = [] # 타깃(시퀀스 다음 글자)을 담을 리스트

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])

print('시퀀스 개수:', len(sentences))

chars = sorted(list(set(text))) # 말뭉치에서 고유한 글자를 담은 리스트
print('고유한 글자:', len(chars))
# chars 리스트에 있는 글자와 글자의 인덱스를 매핑한 딕셔너리
char_indices = dict((char, chars.index(char)) for char in chars)

print('벡터화...')
# 글자를 원-핫 인코딩하여 0과 1의 이진 배열로 바꿈
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i,t,char_indices[char]] = 1
    y[i,char_indices[next_chars[i]]] = 1

시퀀스 개수: 200278
고유한 글자: 58
벡터화...


In [5]:
# 다음 글자를 예측하기 위한 단일 LSTM 모델
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [6]:
# 모델 컴파일 설정하기
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [7]:
# 모델의 예측이 주어졌을 때 새로운 글자를 샘플링하는 함수
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [8]:
# 텍스트 생성 루프
import random
import sys

random.seed(42)
start_index = random.randint(0, len(text)-maxlen-1)

for epoch in range(1,60): # 60 에포크 동안 모델을 훈련
    print('에포크', epoch)
    model.fit(x,y,batch_size=128, epochs=1) # 데이터에서 한 번만 반복해서 모델을 학습
    
    seed_text = text[start_index: start_index+maxlen] # 무작위로 시드 텍스트를 선택
    print('--- 시드 텍스트:"'+seed_text+'"')
    
    for temperature in [0.2,0.5,1.0,1.2]: # 여러 가지 샘플링 온도를 시도
        print('------ 온도:', temperature)
        generated_text = seed_text
        sys.stdout.write(generated_text)
        
        for i in range(400): # 시드 텍스트에서 시작해서 400개의 글자를 생성
            # 지금까지 생성된 글자를 원-핫 인코딩으로 바꿈
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0,t,char_indices[char]] = 1.
                
            # 다음 글자를 샘플링
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            
            generated_text += next_char
            generated_text = generated_text[1:]
            
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

에포크 1

Epoch 1/1
200278/200278 [==============================] - 234s 1ms/step - loss: 1.9690
--- 시드 텍스트:"the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the all the stronger and the manker to the stronger and the some and something and the readness and the spirit is the stronger and the somethings of the resting to the stronger and the same the ensigion to the stronger to the something and the strong and the stronger and somethings the somethings and somethings and all the stronger of the promance of the somethings and all the stronger and the s
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through for the promations of the stand explants of "prome his expection something, the senses of the resting of the something about and solvery of the soul and
spiritated and of somethings of the moraus its also the dasting that a man at perposing to the some and double of the same a soul

takenot, may shear, a power an witjer sepible 
------ 온도: 1.2
the slowly ascending ranks and classes, in which,
through fortunenaniz, alteratizic and lume
assume. this explanation and
socicate to this knowledge and
that only
inglunds as alxykois s.



ssoutiendunaty as he time no spunce,
as fo mpelogative?--whereas hellents,
brought to atonless,
modestry
in be cuess abquted reashs,
by metaphysicrally
a
fornity potenistive even his spirit, the "purpossed and peie meanch in relusing coacle-
nothirr, art does wish , tim
에포크 9
Epoch 1/1
200278/200278 [==============================] - 231s 1ms/step - loss: 1.3877
--- 시드 텍스트:"the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the spirit who superfound the same the work of the strength and the sense of the same and the really the sense of the same the same the same the same the problem the great the sense of the conscious of the same the way and sain the 

through for instufiface in some contait which the taris! finally thereby in attentation of digined
of at "not fawar'. this re"bimom, what i overmert does should contaitions
of tesut to kinds of hard how of the consculves, muspry,
or from to
a facts ethic
and this comparation sought of "higher anlways of refreice-gnowned, a comparely for morality" and called german
hould
to still hoften persises, itself by w
------ 온도: 1.2
the slowly ascending ranks and classes, in which,
through fore better includes a
the se"gious morrowners in nature
predies affect necessari? what should tome of dis only assect of itding, tal'ed hesplateed ay perchive and, about
the deterially how causal good ufferses of mecamer seld is gi
ours once, the
here of
matters in the judgment of the mackivedaboround for the
tormer, bhoughtity, bevo been
judgment do!
end, his ducly, all
molemenmnsm, itlicacume of 
에포크 17
Epoch 1/1
200278/200278 [==============================] - 233s 1ms/step - loss: 1.3296
--- 시드 텍스트:"the sl

c:\users\rladh\anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


 ei, but showen yo s
whand and groubhed by slourit some set
marulacly everander as
so something dictfus at lion
youngs. shald
bringd
and conolive flandand ablewaws relateans. though a lines whither
with others."--mancenor has been contentication, but our own follow celenton 
에포크 18
Epoch 1/1
200278/200278 [==============================] - 255s 1ms/step - loss: 1.3258 1s - los - ETA: 0s - loss: 1.325
--- 시드 텍스트:"the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through form of the sense of the more and the self consists with the strength of the sense of the sense of the spirit and something of the presence of the more and the presence of the man and according to be a such a man and a person and something of the presence of the presence of the sense of the strong as itself of the world of the sense of the strength of the sense of the sense of the sense of the most 
------ 온도: 0.5
the slowly ascending ranks and 

in their estenck that need and can nomed lifulin, and this ch
------ 온도: 1.2
the slowly ascending ranks and classes, in which,
through forgud algen comhaks with old also overwart
as this larding man wilcful heirs to life lesty" wherefore selfivating redurines to anyir radices, grourd:
why cootherfulnes mamilg view--it is morality--the especially, an
histar
aways in europe
pe, is tasng in utill in thole whose might finds word
of youth, in a papiles these mutive, are fortune of reductiosed have as absorutientomogy. in long differend

에포크 26
Epoch 1/1
200278/200278 [==============================] - 224s 1ms/step - loss: 1.2947
--- 시드 텍스트:"the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through foreground and more the senses to the experience, and the personal present consider of the senses the standard of the backgre and more and with the problem of the state of the standard, and such a subjugate the standard 

KeyboardInterrupt: 

### 딥드림

In [4]:
# 사전 훈련된 인셉션 V3 모델 로드하기
from keras.applications import inception_v3
from tensorflow.keras import backend as K

K.set_learning_phase(0) # 모델을 훈련하지 않음 / 이 명령은 모든 훈련 연산을 비활성화함

# 합성곱 기반 층만 사용한 인셉션 V3 네트워크를 만듦 / 사전 훈련된 imageNet 가중치와 함께 모델을 로드
model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

In [5]:
# 딥드림 설정하기
# 층 이름과 계수를 매핑한 딕셔너리 / 최대화하려는 손실에 층의 활성화가 기여할 양을 정함
# 층 이름은 내장된 인셉션 V3 애플리케이션에 하드코딩되어 있는 것 
# model.summary()를 사용하면 모든 층 이름을 확인할 수 있음
layer_contributions = {'mixed2': 0.2, 'mixed3': 3., 'mixed4': 2., 'mixed5': 1.5,}

In [6]:
# 최대화할 손실 정의하기
# 층 이름과 층 객체를 매핑한 딕셔너리를 만듦
layer_dict = dict([(layer.name, layer) for layer in model.layers])
# 손실을 정의하고 각 층의 기여 분을 이 스칼라 변수에 추가
loss = K.variable(0.)
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output # 층의 출력을 얻음
    
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    # 층 특성의 L2 노름 제곱을 손실에 추가 / 이미지 테두리는 제외하고 손실에 추가
    loss.assign(coeff * K.sum(K.square(activation[:,2:-2,2:-2,:])) / scaling)

In [7]:
# 경사 상승법 과정
dream = model.input # 이 텐서는 생성된 딥드림 이미지를 저장

grads = K.gradients(loss, dream)[0] # 손실에 대한 딥드림 이미지의 그래디언트를 계산

grads /= K.maximum(K.mean(K.abs(grads)), 1e-7) # 그래디언트를 정규화 (이 기교가 중요함)

# 주어진 입력 이미지에서 손실과 그래디언트 값을 계산할 케라스 Function 객체를 만듦
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

# 이 함수는 경사 상승법을 여러번 반복하여 수행
def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...', i, '번째 손실:', loss_value)
        x += step * grad_values
    return x

ValueError: None values not supported.

In [8]:
# 연속적인 스케일에 걸쳐 경사 상승법 실행하기
import numpy as np

# 하이퍼파라미터를 바꾸면 새로운 효과가 만들어짐
step = 0.01 # 경사 상승법 단계 크기
num_octave = 3 # 경사 상승법을 실행할 스케일 단계 횟수
octave_scale = 1.4 # 스케일 간의 크기 비율
iterations = 20 # 스케일 단계마다 수행할 경사 상승법 횟수

max_loss = 10. # 손실이 10보다 커지면 이상한 그림이 되는 것을 피하기 위해 경사 상승법 과정을 중지

base_image_path = 'c:/datasets/original_photo_deep_dream.jpg' # 사용할 이미지 경로를 씀

img = preprocess_image(base_image_path) # 기본 이미지를 넘파이 배열로 로드

original_shape = img.shape[1:3]
successive_shapes = [original_shape]

# 경사 상승법을 실행할 스케일 크기를 정의한 튜플의 리스트를 준비
for i in range(1, num_octave):
    shape = tuple([int(dim/(octave_scale**i)) for dim in original_shape])
    successive_shapes.append(shape)
    
successive_shapes = successive_shapes[::-1] # 이 리스트를 크기 순으로 뒤집음

# 이미지의 넘파이 배열을 가장 작은 스케일로 변경
original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0]) 

for shape in successive_shapes:
    print('처리할 이미지 크기:', shape)
    img = resize_img(img, shape) # 딥드림 이미지의 스케일을 키움
    # 경사 상승법을 실행하고 이미지를 변경
    img = gradient_ascent(img, iterations=iterations, step=step, max_loss=max_loss)
    # 작게 줄인 원본 이미지의 스케일을 높임 / 픽셀 경계가 보일 것임
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    # 이 크기에 해당하는 원본 이미지의 고해상도 버전을 계산
    same_size_original = resize_img(original_img, shape)
    # 이 두 이미지의 차이가 스케일을 높였을 때 손실된 디테일
    lost_detail = same_size_original - upscaled_shrunk_original_img
    
    img += lost_detail # 손실된 디테일을 딥드림 이미지에 다시 주입
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale_'+str(shape)+'.png')
    
save_img(img, fname='c:/datasets/final_dream.png')

처리할 이미지 크기: (178, 178)


NameError: name 'gradient_ascent' is not defined

In [2]:
# 유틸리티 함수
import scipy
from keras.preprocessing import image

def resize_img(img, size):
    img = np.copy(img)
    factors = (1, float(size[0]) / img.shape[1], float(size[1]) / img.shape[2], 1)
    return scipy.ndimage.zoom(img, factors, order=1)

def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    image.save_img(fname, pil_img)
    
def preprocess_image(image_path): # 사진을 열고 크기를 줄이고 인셉션 V3가 인식하는 텐서 포맷으로 변환하는 유틸리티 함수
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img

def deprocess_image(x): # 넘파이 배열을 적절한 이미지 포맷으로 변환하는 유틸리티 함수
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1,2,0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x,0,255).astype('uint8')
    return x

Using TensorFlow backend.
c:\users\rladh\anaconda3\envs\py36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\rladh\anaconda3\envs\py36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\rladh\anaconda3\envs\py36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\rladh\anaconda3\envs\py36\lib\site-packag

### 뉴럴 스타일 트랜스퍼

In [9]:
# 변수 초깃값 정의하기
from keras.preprocessing.image import load_img, img_to_array, save_img

target_image_path = 'c:/datasets/portrait.png' # 변환하려는 이미지 경로
style_reference_image_path = 'c:/datasets/popova.jpg' # 스타일 이미지 경로

# 생성된 사진의 차원
width, height = load_img(target_image_path).size
img_height = 400
img_width = int(width * img_height / height)

In [10]:
# 유틸리티 함수
import numpy as np
from keras.applications import vgg19

def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

def deprocess_image(x):
    # ImageNet의 평균 픽셀 값을 더함
    # vgg19.preprocess_input 함수에서 일어나는 변환을 복원
    x[:,:,0] += 103.939
    x[:,:,1] += 116.779
    x[:,:,2] += 123,68
    # 이미지를 BGR에서 RGB로 변환 / 이것도 vgg19.preprocess_input 함수에서 일어나는 변환을 복운하기 위함
    x = x[:,:,::-1]
    x = np.clip(x,0,255).astype('uint8')
    return x

In [ ]:
# 사전 훈련된 VGG19 네트워크를 로딩하고 3개 이미지에 적용하기
from keras import backend as K

target_image = K.constant(preprocess_image(target_image_path))
style_reference_image = K.constant(preprocess_image(style_reference_image_path))
combination_image = K.placeholder((1, img_height, img_width, 3)) # 생성된 이미지를 담을 플레이스홀더

# 3개의 이미지를 하나의 배치로 합침
input_tensor = K.concatenate([target_image, style_reference_image, combination_image], axis=0)

# 세 이미지의 배치를 입력으로 받는 VGG 네트워크를 만듦 / 이 모델은 사전 훈련된 imageNet 가중치를 로드함
model = vgg19.VGG19(input_tensor=input_tensor, weights='imagenet', include_top=False)
print('모델 로드 완료.')